BD SQL & NOSQL Project
======================
### Project based on data from RGPH (recensement général de la population et de l’habitat) in 2014
### DATA SOURCE: [RGPH Website](http://rgphentableaux.hcp.ma)

## Part 04: Interrogating MongoDB Database

In [ ]:
# Install dependencies
!pip install pymongo pandas

In [2]:
# Import libraries
import pymongo
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
#Database Connection
DB_PARAMS = {
    'host':'localhost',
    'port':3306,
    'user':'root',
    'password':'ehtp',
    'database':'ehtp_rgph',
}
mysql_cnx = pymysql.connect(**DB_PARAMS)


In [ ]:

sql_query = """
    SELECT
        pays.libelle AS 'Pays',
        sum( observations_demographie.observation ) AS 'Population municipale' 
    FROM
        observations_demographie
        JOIN communes ON communes.code = observations_demographie.commune_code
        JOIN provinces ON provinces.code = communes.province_code
        JOIN regions ON regions.code = provinces.region_code
        JOIN pays ON pays.code = regions.pays_code 
    WHERE
        observations_demographie.indicateur LIKE '%population munici%' 
    GROUP BY
        pays.libelle
"""
pd.read_sql(sql_query, mysql_cnx)

In [ ]:
# Question 05: Top 10 Populated Communes
sql_query = """
    SELECT
        communes.libelle AS 'Commune',
        sum( observations_demographie.observation ) AS population_municipale 
    FROM
        observations_demographie
        JOIN communes ON communes.code = observations_demographie.commune_code 
    WHERE
        observations_demographie.indicateur LIKE '%population munici%' 
    GROUP BY
        communes.libelle 
    ORDER BY
        population_municipale DESC 
        LIMIT 10;
"""
pd.read_sql(sql_query, mysql_cnx)

In [ ]:
# Question 06: Communes ou of Rabat and Casablanca Region having
# Population greater than 50000, and Unemployment rate lower than 15%
# Female Activity rate greater than 30% and Illiteracy rate lower than 20%
sql_query = """
    SELECT
        communes.code,
        communes.libelle,
        od_s.population,
        oac_s.taux_chomage,
        oaa_s.taux_activite_feminin,
        oe_s.taux_analphabetisme
    FROM
        communes
        JOIN provinces ON provinces.CODE = communes.province_code
        JOIN regions ON regions.CODE = provinces.region_code
        JOIN (
                        SELECT commune_code, SUM( od.observation ) AS population 
                        FROM observations_demographie as od
                        WHERE od.indicateur like '%population municipale%'
                        GROUP BY commune_code 
                    ) AS od_s ON od_s.commune_code = communes.code
        JOIN (
                        SELECT commune_code, AVG( oac.observation ) AS taux_chomage
                        FROM observations_activite as oac
                        WHERE oac.indicateur like '%taux de chômage%'
                        GROUP BY commune_code 
                    ) AS oac_s ON oac_s.commune_code = communes.code
        JOIN (
                        SELECT commune_code, AVG( oaa.observation ) AS taux_activite_feminin
                        FROM observations_activite as oaa
                        WHERE oaa.indicateur like "%f_taux net d'activité%"
                        GROUP BY commune_code 
                    ) AS oaa_s ON oaa_s.commune_code = communes.code
        JOIN (
                        SELECT commune_code, AVG( oe.observation ) AS taux_analphabetisme
                        FROM observations_education as oe
                        WHERE oe.indicateur like "%taux d'analphabétisme%"
                        GROUP BY commune_code 
                    ) AS oe_s ON oe_s.commune_code = communes.code
    WHERE
        regions.id NOT IN (
        '04',
        '06')
    and od_s.population > 50000
    and oac_s.taux_chomage < 15
    and oaa_s.taux_activite_feminin > 30
    and oe_s.taux_analphabetisme < 20
"""
pd.read_sql(sql_query, mysql_cnx)

In [ ]:
# Question 07: Province with the lowerest handicap prevalence rate
sql_query = """
    SELECT
        provinces.CODE,
        provinces.libelle,
        ROUND( AVG( observation ), 2 ) AS taux_prevalence_handicap 
    FROM
        observations_handicap
        JOIN communes ON observations_handicap.commune_code = communes.
        CODE JOIN provinces ON communes.province_code = provinces.CODE 
    GROUP BY
        provinces.CODE,
        provinces.libelle 
    ORDER BY
        taux_prevalence_handicap ASC 
        LIMIT 1;
"""
pd.read_sql(sql_query, mysql_cnx)

In [ ]:
# Question 08: Age Average of first meriage by gender and zones
sql_query = """
    SELECT
        regions.CODE,
        regions.libelle ,
        ROUND(odm_s.age_moyen_1er_mariage_masculin, 2) as age_moyen_1er_mariage_masculin,
        ROUND(odf_s.age_moyen_1er_mariage_feminin, 2) as age_moyen_1er_mariage_feminin,
        ROUND(odu_s.age_moyen_1er_mariage_urbain, 2) as age_moyen_1er_mariage_urbain,
        ROUND(odr_s.age_moyen_1er_mariage_rural, 2) as age_moyen_1er_mariage_rural
    FROM regions
	JOIN (
		SELECT
			provinces.region_code,
			AVG( odm.observation ) age_moyen_1er_mariage_masculin
		FROM
			observations_demographie AS odm
		JOIN communes on communes.code = odm.commune_code
		JOIN provinces on provinces.code = communes.province_code
		WHERE odm.indicateur LIKE '%m_âge moyen au premier mariage%'
		GROUP BY provinces.region_code
	) as odm_s on odm_s.region_code = regions.code

	JOIN (
		SELECT
			provinces.region_code,
			AVG( odf.observation ) age_moyen_1er_mariage_feminin
		FROM
			observations_demographie AS odf
		JOIN communes on communes.code = odf.commune_code
		JOIN provinces on provinces.code = communes.province_code
		WHERE odf.indicateur LIKE '%f_âge moyen au premier mariage%'
		GROUP BY provinces.region_code
	) as odf_s on odf_s.region_code = regions.code

	JOIN (
		SELECT
			provinces.region_code,
			AVG( odu.observation ) age_moyen_1er_mariage_urbain
		FROM
			observations_demographie AS odu
		JOIN communes on communes.code = odu.commune_code
		JOIN provinces on provinces.code = communes.province_code
		WHERE odu.indicateur LIKE '%u%_âge moyen au premier mariage%'
		GROUP BY provinces.region_code
	) as odu_s on odu_s.region_code = regions.code

	JOIN (
		SELECT
			provinces.region_code,
			AVG( odr.observation ) age_moyen_1er_mariage_rural
		FROM
			observations_demographie AS odr
		JOIN communes on communes.code = odr.commune_code
		JOIN provinces on provinces.code = communes.province_code
		WHERE odr.indicateur LIKE '%r%_âge moyen au premier mariage%'
		GROUP BY provinces.region_code
	) as odr_s on odr_s.region_code = regions.code
"""
pd.read_sql(sql_query, mysql_cnx)

In [ ]:
# Question 09: Provinces of The 10 Communes had the lowerest illiteracy rate
sql_query = """
    SELECT
        provinces.libelle AS Province,
        communes.libelle AS Commune,
        ROUND( AVG( observations_education.observation ), 2 ) AS taux_analphabetisme 
    FROM
        observations_education
        JOIN communes ON communes.CODE = observations_education.commune_code
        JOIN provinces ON provinces.CODE = communes.province_code 
    WHERE
        observations_education.indicateur LIKE "%taux d'analphabétisme%" 
    GROUP BY
        communes.libelle,
        provinces.libelle 
    ORDER BY
        taux_analphabetisme DESC 
        LIMIT 10;
"""
pd.read_sql(sql_query, mysql_cnx)

In [ ]:
# Question 10: Top school level for each region
sql_query = """
    SELECT
        rg_e1.region_code,
        rg_e1.region,
        rg_e1.niveau_etude,
        rg_e1.taux 
    FROM
        (
        SELECT
            regions.CODE AS region_code,
            regions.libelle AS region,
            SUBSTR( observations_education.indicateur, 5 ) AS niveau_etude,
            ROUND( AVG( observations_education.observation ), 2 ) AS taux 
        FROM
            observations_education
            JOIN communes ON communes.CODE = observations_education.commune_code
            JOIN provinces ON provinces.CODE = communes.province_code
            JOIN regions ON regions.CODE = provinces.region_code 
        WHERE
            observations_education.indicateur LIKE "%niveau d'études%" 
        GROUP BY
            niveau_etude,
            regions.CODE,
            regions.libelle 
        ORDER BY
            regions.CODE,
            taux DESC 
        ) rg_e1
    JOIN (
        SELECT
            rg_i.region_code,
            MAX( rg_i.taux ) AS taux_max 
        FROM
            (
            SELECT
                regions.CODE AS region_code,
                regions.libelle AS region,
                SUBSTR( observations_education.indicateur, 5 ) AS niveau_etude,
                ROUND( AVG( observations_education.observation ), 2 ) AS taux 
            FROM
                observations_education
                JOIN communes ON communes.CODE = observations_education.commune_code
                JOIN provinces ON provinces.CODE = communes.province_code
                JOIN regions ON regions.CODE = provinces.region_code 
            WHERE
                observations_education.indicateur LIKE "%niveau d'études%" 
            GROUP BY
                niveau_etude,
                regions.CODE,
                regions.libelle 
            ORDER BY
                regions.CODE,
                taux DESC 
        ) AS rg_i 
    GROUP BY
        rg_i.region_code 
    ) AS rg_e2 ON rg_e2.region_code = rg_e1.region_code 
    AND rg_e2.taux_max = rg_e1.taux
"""
pd.read_sql(sql_query, mysql_cnx)

In [ ]:
# Question 11: Provinces where Tachelhit being used more than Tamazaghit
#              and both greater than 50
sql_query = """
    SELECT
        provinces.`code`,
        provinces.libelle,
        olc.tachalhit,
        olz.tamazight 
    FROM
        provinces
        JOIN (
        SELECT
            communes.province_code,
            ROUND( AVG( ol_tc.observation ), 2 ) tachalhit 
        FROM
            observations_langue AS ol_tc
            JOIN communes ON communes.CODE = ol_tc.commune_code 
        WHERE
            ol_tc.indicateur LIKE '%tachelhit%' 
        GROUP BY
            communes.province_code 
        ) AS olc ON olc.province_code = provinces.
        CODE JOIN (
        SELECT
            communes.province_code,
            ROUND( AVG( ol_tz.observation ), 2 ) tamazight 
        FROM
            observations_langue AS ol_tz
            JOIN communes ON communes.CODE = ol_tz.commune_code 
        WHERE
            ol_tz.indicateur LIKE '%tamazight%' 
        GROUP BY
            communes.province_code 
        ) AS olz ON olz.province_code = provinces.CODE 
    WHERE
        olc.tachalhit + olz.tamazight > 50 
        AND olc.tachalhit > olz.tamazight
"""
pd.read_sql(sql_query, mysql_cnx)

In [ ]:
# Question 12: Nomber of Provinces uses Satellite dish more than Fridge
sql_query = """
    SELECT
        COUNT(*) as 'Nombre des Provinces'
    FROM
        provinces
        JOIN (
            SELECT
                communes.province_code,
                ROUND( AVG( ol_p.observation ), 2 ) parabole 
            FROM
                observations_habitat AS ol_p
                JOIN communes ON communes.CODE = ol_p.commune_code 
            WHERE
                ol_p.indicateur LIKE '%equipements ménagers_parabole%' 
            GROUP BY
            communes.province_code 
        ) AS uh_p ON uh_p.province_code = provinces.CODE
        
        JOIN (
            SELECT
                communes.province_code,
                ROUND( AVG( ol_r.observation ), 2 ) refrigerateur
            FROM
                observations_habitat AS ol_r
                JOIN communes ON communes.CODE = ol_r.commune_code 
            WHERE
                ol_r.indicateur LIKE '%equipements ménagers_réfrigérateur%' 
            GROUP BY
            communes.province_code 
        ) AS uh_r ON uh_r.province_code = provinces.CODE
where uh_p.parabole > uh_r.refrigerateur
"""
pd.read_sql(sql_query, mysql_cnx)